# Outliers detection classic methods

We use the following approaches:
* (always) mean ± \[3] SD 
* (always) meadian ± \[3] MAD
* (always) IQR x \[1.5] 

--------------------

**Input**: 
* **routine DHIS2** data (formatted and aligned)
    * from Dataset "**snt-dhis2-formatted**", `XXX_routine.parquet`

**Output**: 
All outputs saved to Dataset "**snt-outliers-detection**", with the following .parquet files:
* **1 comprehensive table** with flags for all outliers methods selected by the user
    *  cols: YEAR, MONTH, ADM1_ID, ADM2_ID, OU_ID, INDICATOR, VALUE, **OUTLIER_METHOD_X**, **OUTLIER_METHOD_Y**, **OUTLIER_METHOD_Z** ...
    *  Filename: `XXX_flagged_outliers_allmethods.parquet`
* **3-5 individual tables** (separated files), each containig the flags for _a given method only_
    *   cols: YEAR, MONTH, ADM1_ID, ADM2_ID, OU_ID, INDICATOR, VALUE, **OUTLIER_METHOD_X**
    *   Filename: `XXX_outlier_<method_name>.parquet`
* 🐘 **Table** in ws **Database** with added cols needed for 📊 **Shiny App: SNT Outliers Explorer**
    *   cols: YEAR, MONTH, ADM1_ID, ADM2_ID, OU_ID, INDICATOR, VALUE, OUTLIER_METHOD_X, OUTLIER_METHOD_Y, OUTLIER_METHOD_Z, **ADM1_NAME**, **ADM2_NAME**, **OU_NAME**, **DATE**
    *   Table name: `flagged_outliers_allmethods_name_date`

---------------------

In [ ]:
# Parameters
# DEVIATION_MEAN <- 3
# DEVIATION_MEDIAN <- 3
# DEVIATION_IQR <- 1.5  

## 1. Setup

In [ ]:
# Project folders
ROOT_PATH <- "~/workspace"  
CODE_PATH <- file.path(ROOT_PATH, 'code') 
CONFIG_PATH <- file.path(ROOT_PATH, 'configuration')
DATA_PATH <- file.path(ROOT_PATH, 'data')

# Load utils
source(file.path(CODE_PATH, "snt_utils.r"))

# Load libraries 
required_packages <- c( "data.table", "arrow", "tidyverse", "jsonlite", "DBI", "RPostgres", "reticulate", "glue", "zoo")
install_and_load(required_packages)

# Environment variables
Sys.setenv(PROJ_LIB = "/opt/conda/share/proj")
Sys.setenv(GDAL_DATA = "/opt/conda/share/gdal")
Sys.setenv(RETICULATE_PYTHON = "/opt/conda/bin/python")

# Load OpenHEXA sdk
openhexa <- import("openhexa.sdk")

### 1.1. Validate parameters

In [ ]:
if (!exists("DEVIATION_MEAN")) DEVIATION_MEAN <- 3
if (!exists("DEVIATION_MEDIAN")) DEVIATION_MEDIAN <- 3
if (!exists("DEVIATION_IQR")) DEVIATION_IQR <- 1.5 

### 1.2. Load and check `SNT_config` file

In [ ]:
# Load SNT config
config_json <- tryCatch({ fromJSON(file.path(CONFIG_PATH, "SNT_config.json")) },
    error = function(e) {
        msg <- glue("[ERROR] Error while loading configuration {conditionMessage(e)}")
        log_msg(msg)
        stop(msg)
    })

log_msg(glue("SNT configuration loaded from  : {file.path(CONFIG_PATH, 'SNT_config.json')}"))

In [ ]:
# Check SNT configuration 
snt_config_mandatory <- c("COUNTRY_CODE", "DHIS2_ADMINISTRATION_1", "DHIS2_ADMINISTRATION_2") 
for (conf in snt_config_mandatory) {
    if (is.null(config_json$SNT_CONFIG[[conf]])) {
        msg <- paste("Missing configuration input:", conf)
        log_msg(msg)
        stop(msg)
    }
}

COUNTRY_CODE <- config_json$SNT_CONFIG$COUNTRY_CODE
ADMIN_1 <- toupper(config_json$SNT_CONFIG$DHIS2_ADMINISTRATION_1)
ADMIN_2 <- toupper(config_json$SNT_CONFIG$DHIS2_ADMINISTRATION_2)
DHIS2_INDICATORS <- names(config_json$DHIS2_DATA_DEFINITIONS$DHIS2_INDICATOR_DEFINITIONS)
fixed_cols = c('PERIOD', 'YEAR', 'MONTH', 'ADM1_ID', 'ADM2_ID', 'OU_ID')

## 2. Load Data

### 2.1. **Routine** data (DHIS2) 

Formatted & aggregated data stored in OpenHEXA Dataset "**SNT_DHIS2_FORMATTED**"

In [ ]:
# Load file from dataset (formatting)
dataset_name <- config_json$SNT_DATASET_IDENTIFIERS$DHIS2_DATASET_FORMATTED
dhis2_routine <- tryCatch({ get_latest_dataset_file_in_memory(dataset_name, paste0(COUNTRY_CODE, "_routine.parquet")) }, 
    error = function(e) {
        msg <- glue("[ERROR] Error while loading DHIS2 routine data file for {COUNTRY_CODE} : {conditionMessage(e)}")  # log error message
        log_msg(msg)
        stop(msg)
})

log_msg(glue("DHIS2 routine data loaded from dataset : {dataset_name}"))
log_msg(glue("DHIS2 routine data loaded has dimensions: {nrow(dhis2_routine)} rows, {ncol(dhis2_routine)} columns."))
print(dim(dhis2_routine))
head(dhis2_routine, 2)

🔍 **Assert indicators are present**

In [ ]:
# Raise an error if any of DHIS2_INDICATORS are not present in the dhis2 routine data.
for (ind in DHIS2_INDICATORS) {
    if (!(ind %in% colnames(dhis2_routine))) {
        msg <- paste("[ERROR] Missing indicator column in routine data: ", ind)
        log_msg(msg)
        stop(msg)
    }
}

## 3. Outliers Detection

### 3.1. Transform routine data  

* **Pivot longer***: cols become rows

In [ ]:
dhis2_routine_long <- dhis2_routine %>%
    select(all_of(c("ADM1_ID", "ADM2_ID", "OU_ID", "PERIOD", DHIS2_INDICATORS))) %>%
    pivot_longer(cols = all_of(DHIS2_INDICATORS), names_to = "INDICATOR", values_to = "VALUE")

print(dim(dhis2_routine_long))
head(dhis2_routine_long, 2)

🔍 **Remove duplicated values**

In [ ]:
# check if there are any duplicates
duplicated <- dhis2_routine_long %>%
  group_by(ADM1_ID, ADM2_ID, OU_ID, PERIOD, INDICATOR) %>%
  summarise(n = dplyr::n(), .groups= "drop") %>%
  filter(n > 1L)

# Remove dups
if (nrow(duplicated) > 0) {
    log_msg(glue("Removing {nrow(duplicated)} duplicated values."))
    dhis2_routine_long <- dhis2_routine_long %>%
        distinct(ADM1_ID, ADM2_ID, OU_ID, PERIOD, INDICATOR, .keep_all = TRUE)
    head(duplicated)
}

### 3.2. Calculate **summary stats**
At `OU_ID` (Health Facility) x `INDICATOR`, calculate:
* mean
* median
* SD
* MAD
* Q1 (25th)
* Q3 (75th).

In [ ]:
# stats
dhis2_routine_stats <- dhis2_routine_long %>%
    group_by(across(all_of(c("ADM1_ID", "ADM2_ID", "OU_ID", "INDICATOR")))) %>%  # , YEAR?
    mutate(
        # n = n(), # added for inspection  
        # n_positive = length(na.omit(VALUE)), # ⚠️ 2025-08-26: added for inspection  
        mean = ceiling(mean(VALUE, na.rm = TRUE)),
        median = ceiling(median(VALUE, na.rm = TRUE)),
        sd = ceiling(sd(VALUE, na.rm = TRUE)),
        mad = ceiling(mad(VALUE, constant = 1, na.rm = TRUE)), # 🚨 scale factor: `constant = 1` (default `constant = 1.4826`) 
        q1 = ceiling(quantile(VALUE, 0.25, na.rm = TRUE)), 
        q3 = ceiling(quantile(VALUE, 0.75, na.rm = TRUE))
      ) %>% 
      ungroup() 

dim(dhis2_routine_stats)
head(dhis2_routine_stats, 2)

### 3.3. Flag outlier values: use the 3 **classic** methods

Flagging from *all* the 3 methods in the same table (each method is a column).

In [ ]:
# just renaming the method column name
DEVIATION_IQR_NAME <- gsub("\\.", "_", as.character(DEVIATION_IQR))

# outliers detection
dhis2_routine_outliers <- dhis2_routine_stats %>% 
    mutate(
        mean_lower_bound = mean - DEVIATION_MEAN * sd, 
        mean_upper_bound = mean + DEVIATION_MEAN * sd,
        !!sym(glue("OUTLIER_MEAN{DEVIATION_MEAN}SD")) := if_else(
          VALUE < mean_lower_bound | VALUE > mean_upper_bound,
          TRUE, # = outlier
          FALSE
        )) %>% 
      mutate(
        median_lower_bound = median - DEVIATION_MEDIAN * mad,
        median_upper_bound = median + DEVIATION_MEDIAN * mad,
        !!sym(glue("OUTLIER_MEDIAN{DEVIATION_MEDIAN}MAD")) := if_else(
          VALUE < median_lower_bound | VALUE > median_upper_bound,
          TRUE,
          FALSE
        )) %>% 
      mutate(
        iqr = (q3 - q1) * DEVIATION_IQR,
        iqr_lower_bound = q1 - iqr,
        iqr_upper_bound = q3 + iqr,
        !!sym(glue("OUTLIER_IQR{DEVIATION_IQR_NAME}")) := if_else(
          VALUE < iqr_lower_bound | VALUE > iqr_upper_bound,
          TRUE,
          FALSE
        )) 

outlier_cols = dhis2_routine_outliers %>% select(starts_with("OUTLIER_")) %>% names()
log_msg(paste0("Calculated columns : ", paste(outlier_cols, collapse=", ")))

print(dim(dhis2_routine_outliers))
head(dhis2_routine_outliers, 2)

### 3.4. Flag `NA`s as non-outliers: 

This overall eventually makes all `VALUE == 0` into not-outlier, because upstream all `VALUE == 0` where replaced with `NA` to be ignored by the summary stats that defined the bundaries for outliers (mean, median, mad, sd).

In [ ]:
dhis2_routine_outliers <- dhis2_routine_outliers %>%
  mutate(across(starts_with("OUTLIER_"), ~ if_else(is.na(.x), FALSE, .x)))

In [ ]:
# dhis2_routine_outliers$OUTLIER_MEAN3SD %>% sum()
# dhis2_routine_outliers$OUTLIER_MEDIAN3MAD %>% sum()
# dhis2_routine_outliers$OUTLIER_IQR1_5 %>% sum()

### 3.5. Select outliers columns only

This table contains **all values**, **flagged** (bool) based on each (and all) outliers detection method used.

**Structure**: contains always the same fixed set of cols (`fixed_cols`, INDICATOR, VALUE) + **1 col for each** implemented **outliers detection method** <br>
(so table keeps growing as cols are added to the right end)

In [ ]:
# Select outlier columns
dhis2_routine_outliers_selection <- dhis2_routine_outliers %>% 
    mutate(
        YEAR = as.integer(substr(PERIOD, 1, 4)),
        MONTH = as.integer(substr(PERIOD, 5, 6))) %>%
    select(any_of(c(fixed_cols, "INDICATOR", "VALUE")), starts_with("OUTLIER_"))

print(dim(dhis2_routine_outliers_selection))
head(dhis2_routine_outliers_selection, 2) # <----------------------------- OUTLIERS TABLE

In [ ]:
# log detection results
outlier_columns <- colnames(dhis2_routine_outliers_selection)[startsWith(colnames(dhis2_routine_outliers_selection), "OUTLIER_")]
for (outliers_col in outlier_columns) {
    if (substr(outliers_col, 1, 12) == "OUTLIER_MEAN") outlier_method <- glue("{DEVIATION_MEAN}*SD deviation from mean")
    if (substr(outliers_col, 1, 14) == "OUTLIER_MEDIAN") outlier_method <- glue("{DEVIATION_MEDIAN}*MAD deviation from median")
    if (substr(outliers_col, 1, 11) == "OUTLIER_IQR") outlier_method <- glue("IQR {DEVIATION_IQR}")
    nr_of_outliers <- nrow(dhis2_routine_outliers_selection[dhis2_routine_outliers_selection[outliers_col] == TRUE, ])
    perc_outliers <- nr_of_outliers/nrow(dhis2_routine_outliers_selection) * 100
    log_msg(glue("Using {outlier_method} method, {nr_of_outliers} outliers were identified ({sprintf('%.3f', perc_outliers)} % of data points).")) 
}

## 4. Routine data imputation

Generate a routine data version with imputed values for each method.

### 4.1. Impute values to outliers

Compute moving average column ([-1, +1] points window) to be used as imputation value.

In [ ]:
# select columns (these names are dinamycally chosen based on the input, shouldn't be just standard?)
mean_column <- grep("^OUTLIER_MEAN", colnames(dhis2_routine_outliers_selection), value = TRUE)[1]
median_column <- grep("^OUTLIER_MEDIAN", colnames(dhis2_routine_outliers_selection), value = TRUE)[1]
iqr_column <- grep("^OUTLIER_IQR", colnames(dhis2_routine_outliers_selection), value = TRUE)[1]

In [ ]:
# Define helper function to compute moving average for an outlier column
start_time <- Sys.time()

impute_outliers_dt <- function(dt, outlier_col) {
    dt <- as.data.table(dt)  # transform to datatable
    setorder(dt, ADM1_ID, ADM2_ID, OU_ID, INDICATOR, PERIOD)  
    dt[, TO_IMPUTE := fifelse(get(outlier_col) == TRUE, NA_real_, VALUE)]  # Compute TO_IMPUTE column
    
    # Fast rolling mean by group
    dt[, MOVING_AVG := frollapply(TO_IMPUTE, n = 3, FUN = function(x) ceiling(mean(x, na.rm = TRUE)), align = "center"), 
     by = .(ADM1_ID, ADM2_ID, OU_ID, INDICATOR)]
    
    dt[, VALUE_IMPUTED := fifelse(is.na(TO_IMPUTE), MOVING_AVG, TO_IMPUTE)]  
    dt[, c("TO_IMPUTE") := NULL]  # clean up "MOVING_AVG"
                                  
    return(as.data.frame(copy(dt)))
}

In [ ]:
# impute outliers
log_msg("Running imputation for outliers detected using Mean method.")
dhis2_routine_outliers_mean_imputed <- impute_outliers_dt(dhis2_routine_outliers_selection, mean_column)

log_msg("Running imputation for outliers detected using Median (MAD) method.")
dhis2_routine_outliers_median_imputed <- impute_outliers_dt(dhis2_routine_outliers_selection, median_column)

log_msg("Running imputation for outliers detected using IQR method.")
dhis2_routine_outliers_iqr_imputed <- impute_outliers_dt(dhis2_routine_outliers_selection, iqr_column)
                                
Sys.time() - start_time

In [ ]:
# dhis2_routine_outliers_mean_imputed$OUTLIER_MEAN3SD %>% sum()
# dhis2_routine_outliers_median_imputed$OUTLIER_MEDIAN3MAD %>% sum()
# dhis2_routine_outliers_iqr_imputed$OUTLIER_IQR1_5 %>% sum()

### 4.2. Routine data imputed tables

We create two versions of each table `imputation` and `removed` (outliers)

In [ ]:
# get names from routine (This cleaning only applies to DRC names)
pyramid_names <- dhis2_routine %>% 
    distinct(ADM1_NAME, ADM1_ID, ADM2_NAME, ADM2_ID, OU_ID, OU_NAME) %>%
    # Simpify strings 
    mutate(
        ADM1_NAME = stringr::str_trim(str_remove_all(ADM1_NAME, "^[A-Z]{2}| PROVINCE")),
        ADM2_NAME = stringr::str_trim(str_remove_all(ADM2_NAME, "^[A-Z]{2}| ZONE DE SANTE"))
    )

In [ ]:
# Define helper function to format both versions 
format_routine_data_selection <- function(df, outlier_column, remove = FALSE) {
  
  # Decide which rows to keep
  if (remove) df <- df %>% filter(!.data[[outlier_column]])
  
  df %>%
    select(-VALUE) %>%
    rename(VALUE = VALUE_IMPUTED) %>%
    select(all_of(fixed_cols), INDICATOR, VALUE) %>%  # global: fixed_cols
    mutate(VALUE = ifelse(is.nan(VALUE), NA_real_, VALUE)) %>%
    pivot_wider(names_from = "INDICATOR", values_from = "VALUE") %>%
    left_join(pyramid_names, by = c("ADM1_ID", "ADM2_ID", "OU_ID"))
}

In [ ]:
# format mean tables
dhis2_routine_mean_imputed <- format_routine_data_selection(dhis2_routine_outliers_mean_imputed, mean_column)
dhis2_routine_mean_removed <- format_routine_data_selection(dhis2_routine_outliers_mean_imputed, mean_column, remove=TRUE)

In [ ]:
# format median tables
dhis2_routine_median_imputed <- format_routine_data_selection(dhis2_routine_outliers_median_imputed, median_column)
dhis2_routine_median_removed <- format_routine_data_selection(dhis2_routine_outliers_median_imputed, median_column, remove=TRUE)

In [ ]:
# format iqr tables
dhis2_routine_iqr_imputed <- format_routine_data_selection(dhis2_routine_outliers_iqr_imputed, iqr_column)
dhis2_routine_iqr_removed <- format_routine_data_selection(dhis2_routine_outliers_iqr_imputed, iqr_column, remove=TRUE)

## 5. Export Output tables

Export tables as `.parquet` files to `data/` folder.

In [ ]:
output_path <- file.path(DATA_PATH , "dhis2", "outliers_imputation")
                         
# outliers table 
routine_outliers_db_table <- dhis2_routine_outliers_selection %>% 
    mutate(DATE = make_date(year = YEAR, month = MONTH, day = 1L)) %>%
    left_join(pyramid_names, by = c("ADM1_ID", "ADM2_ID", "OU_ID")) 
write_parquet(routine_outliers_db_table, file.path(output_path, paste0(COUNTRY_CODE, "_routine_outliers-classic_detection.parquet")))
log_msg(glue("Outliers detection table saved."))

In [ ]:
# Mean table results
write_parquet(dhis2_routine_mean_imputed, file.path(output_path, paste0(COUNTRY_CODE, "_routine_outliers-mean_imputed.parquet")))
write_parquet(dhis2_routine_mean_removed, file.path(output_path, paste0(COUNTRY_CODE, "_routine_outliers-mean_removed.parquet")))
log_msg(glue("Mean deviation detection results saved."))

In [ ]:
# Median table results
write_parquet(dhis2_routine_median_imputed, file.path(output_path, paste0(COUNTRY_CODE, "_routine_outliers-median_imputed.parquet")))
write_parquet(dhis2_routine_median_removed, file.path(output_path, paste0(COUNTRY_CODE, "_routine_outliers-median_removed.parquet")))
log_msg(glue("Median deviation detection results saved."))

In [ ]:
# Mean table results
write_parquet(dhis2_routine_iqr_imputed, file.path(output_path, paste0(COUNTRY_CODE, "_routine_outliers-iqr_imputed.parquet")))
write_parquet(dhis2_routine_iqr_removed, file.path(output_path, paste0(COUNTRY_CODE, "_routine_outliers-iqr_removed.parquet")))
log_msg(glue("IQR detection results saved."))

In [ ]:
log_msg(glue("Results saved under: {output_path}"))    